# FLDetector for Fashion MNIST


In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

/scratch/gypsum-gpu065/6672123/ipykernel_361451/912229180.py:1: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


In [2]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
from torchvision import models
import torch.optim as optim
from torch.autograd import Variable
import numpy as np
import random
import copy
import time
from functools import reduce
from torchsummary import summary

import os
import sys
import pickle
sys.path.insert(0,'./utils/')
from logger import *
from eval import *
from misc import *

from sklearn.metrics import roc_auc_score
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from collections import defaultdict

from SGD import *
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print(device)

/work/vshejwalkar_umass_edu/anaconda/envs/myenv/lib/python3.10/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: /work/vshejwalkar_umass_edu/anaconda/envs/myenv/lib/python3.10/site-packages/torchvision/image.so: undefined symbol: _ZN2at4_ops19empty_memory_format4callEN3c108ArrayRefIlEENS2_8optionalINS2_10ScalarTypeEEENS5_INS2_6LayoutEEENS5_INS2_6DeviceEEENS5_IbEENS5_INS2_12MemoryFormatEEE
  warn(f"Failed to load image Python extension: {e}")


cuda


In [3]:
transform = transforms.Compose([transforms.ToTensor()])
trainset = torchvision.datasets.FashionMNIST(root='./data', train=True, download=True, transform=transform)
testset = torchvision.datasets.FashionMNIST(root='./data', train=False, download=True, transform=transform)

In [4]:
def sample_dirichlet_train_data(trainset, no_participants, alpha=0.9, force=False):
        """
            Input: Number of participants and alpha (param for distribution)
            Output: A list of indices denoting data in CIFAR training set.
            Requires: cifar_classes, a preprocessed class-indice dictionary.
            Sample Method: take a uniformly sampled 10-dimension vector as parameters for
            dirichlet distribution to sample number of images in each class.
        """
        if not os.path.exists('./dirichlet_a_%.1f_nusers_%d.pkl'%(alpha, no_participants)) or force:
            print('generating participant indices for alpha %.1f'%alpha)
            np.random.seed(0)
            cifar_classes = {}
            for ind, x in enumerate(trainset):
                _, label = x
                if label in cifar_classes:
                    cifar_classes[label].append(ind)
                else:
                    cifar_classes[label] = [ind]

            per_participant_list = defaultdict(list)
            no_classes = len(cifar_classes.keys())
            for n in range(no_classes):
                random.shuffle(cifar_classes[n])
                sampled_probabilities = len(cifar_classes[n]) * np.random.dirichlet(
                    np.array(no_participants * [alpha]))
                for user in range(no_participants):
                    no_imgs = int(round(sampled_probabilities[user]))
                    sampled_list = cifar_classes[n][:min(len(cifar_classes[n]), no_imgs)]
                    per_participant_list[user].extend(sampled_list)
                    cifar_classes[n] = cifar_classes[n][min(len(cifar_classes[n]), no_imgs):]
            with open('./dirichlet_a_%.1f_nusers_%d.pkl'%(alpha, no_participants), 'wb') as f:
                pickle.dump(per_participant_list, f)
        else:
            per_participant_list = pickle.load(open('./dirichlet_a_%.1f_nusers_%d.pkl'%(alpha, no_participants), 'rb'))
            
        return per_participant_list

In [5]:
def get_client_train_data(trainset, num_workers=100, bias=0.5):
    np.random.seed(42)
    bias_weight = bias
    other_group_size = (1 - bias_weight) / 9.
    worker_per_group = num_workers / 10

    each_worker_data = [[] for _ in range(num_workers)]
    each_worker_label = [[] for _ in range(num_workers)]
    
    for i, (x, y) in enumerate(trainset):
        # assign a data point to a group
        upper_bound = (y) * (1 - bias_weight) / 9. + bias_weight
        lower_bound = (y) * (1 - bias_weight) / 9.
        rd = np.random.random_sample()

        if rd > upper_bound:
            worker_group = int(np.floor((rd - upper_bound) / other_group_size) + y + 1)
        elif rd < lower_bound:
            worker_group = int(np.floor(rd / other_group_size))
        else:
            worker_group = y

        rd = np.random.random_sample()
        selected_worker = int(worker_group * worker_per_group + int(np.floor(rd * worker_per_group)))
        
        if not len(each_worker_data[selected_worker]):
            each_worker_data[selected_worker] = x[None, :]
        else:
            each_worker_data[selected_worker]= torch.concat((each_worker_data[selected_worker], x[None, :]))
        
        each_worker_label[selected_worker].append(y)
    
    each_worker_tr_data = [[] for _ in range(num_workers)]
    each_worker_tr_label = [[] for _ in range(num_workers)]
    each_worker_val_data = [[] for _ in range(num_workers)]
    each_worker_val_label = [[] for _ in range(num_workers)]
    each_worker_te_data = [[] for _ in range(num_workers)]
    each_worker_te_label = [[] for _ in range(num_workers)]

    for i in range(num_workers):
        w_len = len(each_worker_data[i])
        len_tr = int(5 * w_len / 7)
        len_val = int(1 * w_len / 7)
        len_te = w_len - (len_tr + len_val)
        # tr_idx = np.random.choice(w_len, len_tr, replace=False)
        # te_idx = np.delete(np.arange(w_len), tr_idx)
        w = np.arange(w_len)
        np.random.shuffle(w)
        tr_idx, val_idx, te_idx = w[:len_tr], w[len_tr : (len_tr + len_val)], w[-len_te:]
        each_worker_tr_data[i] = each_worker_data[i][tr_idx]
        each_worker_tr_label[i] = torch.Tensor(each_worker_label[i])[tr_idx]
        each_worker_val_data[i] = each_worker_data[i][val_idx]
        each_worker_val_label[i] = torch.Tensor(each_worker_label[i])[val_idx]
        each_worker_te_data[i] = each_worker_data[i][te_idx]
        each_worker_te_label[i] = torch.Tensor(each_worker_label[i])[te_idx]
        
    global_test_data = torch.concat(each_worker_te_data)
    global_test_label = torch.concat(each_worker_te_label)
    del each_worker_data, each_worker_label
    return each_worker_tr_data, each_worker_tr_label, each_worker_val_data, each_worker_val_label, each_worker_te_data, each_worker_te_label, global_test_data, global_test_label

In [6]:
def get_client_data_dirichlet(trainset, num_workers, alpha=1, force=False):
    per_participant_list = sample_dirichlet_train_data(trainset, num_workers, alpha=alpha, force=force)
    
    each_worker_data = [[] for _ in range(num_workers)]
    each_worker_label = [[] for _ in range(num_workers)]
    
    each_worker_te_data = [[] for _ in range(num_workers)]
    each_worker_te_label = [[] for _ in range(num_workers)]
    
    np.random.seed(0)
    
    for worker_idx in range(len(per_participant_list)):
        w_indices = np.array(per_participant_list[worker_idx])
        w_len = len(w_indices)
        len_tr = int(6*w_len/7)
        len_te = w_len - len_tr
        tr_idx = np.random.choice(w_len, len_tr, replace=False)
        te_idx = np.delete(np.arange(w_len), tr_idx)
        
        for idx in w_indices[tr_idx]:
            each_worker_data[worker_idx].append(trainset[idx][0])
            each_worker_label[worker_idx].append(trainset[idx][1])
        each_worker_data[worker_idx] = torch.stack(each_worker_data[worker_idx])
        each_worker_label[worker_idx] = torch.Tensor(each_worker_label[worker_idx]).long()
        
        for idx in w_indices[te_idx]:
            each_worker_te_data[worker_idx].append(trainset[idx][0])
            each_worker_te_label[worker_idx].append(trainset[idx][1])
        each_worker_te_data[worker_idx] = torch.stack(each_worker_te_data[worker_idx])
        each_worker_te_label[worker_idx] = torch.Tensor(each_worker_te_label[worker_idx]).long()
    
    global_test_data = torch.concat(each_worker_te_data)
    global_test_label = torch.concat(each_worker_te_label)
    
    return each_worker_data, each_worker_label, each_worker_te_data, each_worker_te_label, global_test_data, global_test_label


In [7]:
def tr_mean(all_updates, n_attackers):
    sorted_updates = torch.sort(all_updates, 0)[0]
    out = torch.mean(sorted_updates[n_attackers:-n_attackers], 0) if n_attackers else torch.mean(sorted_updates,0)
    return out

In [8]:
class cnn(nn.Module):
    def __init__(self):
        super(cnn, self).__init__()
        self.conv1 = nn.Conv2d(1, 30, 3)
        self.pool1 = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(30, 50, 3)
        self.pool2 = nn.MaxPool2d(2,2)
        self.fc1 = nn.Linear(1250, 200)
        self.fc2 = nn.Linear(200, 10)

    def forward(self, x):
#         print(x.shape)
        x = self.pool1(F.relu(self.conv1(x)))
#         print(x.shape)
        x = self.pool2(F.relu(self.conv2(x)))
#         print(x.shape)
        x = x.view(x.size(0), -1)
#         print(x.shape)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# Good FedAvg baseline Fashion MNIST + Fang distribution + 80 clients

In [9]:
from torch.nn.utils import parameters_to_vector, vector_to_parameters

def train(train_data, labels, model, optimizer, batch_size=20):
    model.train()
    criterion = nn.CrossEntropyLoss()
    losses = AverageMeter()
    top1 = AverageMeter()
    top5 = AverageMeter()

    len_t = (len(train_data) // batch_size)
    if len(train_data)%batch_size:
        len_t += 1

    r=np.arange(len(train_data))
    np.random.shuffle(r)
    
    train_data = train_data[r]
    labels = labels[r]
    
    for ind in range(len_t):

        inputs = train_data[ind * batch_size:(ind + 1) * batch_size]
        targets = labels[ind * batch_size:(ind + 1) * batch_size]

        inputs, targets = inputs.cuda(), targets.cuda()

        inputs, targets = torch.autograd.Variable(inputs), torch.autograd.Variable(targets)
        outputs = model(inputs)
        loss = criterion(outputs, targets)

        # measure accuracy and record loss
        prec1, prec5 = accuracy(outputs.data, targets.data, topk=(1, 5))
        losses.update(loss.item(), inputs.size(0))
        top1.update(prec1.item(), inputs.size(0))
        top5.update(prec5.item(), inputs.size(0))

        # compute gradient and do SGD step
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    return (losses.avg, top1.avg)


def test(test_data, labels, model, criterion, use_cuda, debug_='MEDIUM', batch_size=64):
    losses = AverageMeter()
    top1 = AverageMeter()
    top5 = AverageMeter()

    model.eval()
    len_t = (len(test_data) // batch_size)
    if len(test_data)%batch_size:
        len_t += 1

    with torch.no_grad():
        for ind in range(len_t):
            # measure data loading time
            inputs = test_data[ind * batch_size:(ind + 1) * batch_size]
            targets = labels[ind * batch_size:(ind + 1) * batch_size]

            inputs, targets = inputs.cuda(), targets.cuda()

            inputs, targets = torch.autograd.Variable(inputs), torch.autograd.Variable(targets)
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            
            prec1, prec5 = accuracy(outputs.data, targets.data, topk=(1, 5))
            losses.update(loss.item(), inputs.size(0))
            top1.update(prec1.item(), inputs.size(0))
            top5.update(prec5.item(), inputs.size(0))

    return (losses.avg, top1.avg)

# NDSS attack on Dir + TrMean

In [11]:
def ndss21_attack_trmean(all_updates, n_attackers, dev_type='sign', threshold=5.0, threshold_diff=1e-5, fast_ndss=False):
    
    model_re = torch.mean(all_updates, 0)
    
    if dev_type == 'sign':
        deviation = torch.sign(model_re)
    elif dev_type == 'unit_vec':
        deviation = model_re / torch.norm(model_re)  # unit vector, dir opp to good dir
    elif dev_type == 'std':
        deviation = torch.std(all_updates, 0)

    if fast_ndss:
        return (model_re - threshold * deviation)

    lamda = torch.Tensor([threshold]).to(device)  # compute_lambda_our(all_updates, model_re, n_attackers)

    threshold_diff = threshold_diff
    prev_loss = -1
    lamda_fail = lamda
    lamda_succ = 0

    while torch.abs(lamda_succ - lamda) > threshold_diff:
        mal_update = (model_re - lamda * deviation)
        mal_updates = torch.stack([mal_update] * n_attackers)
        mal_updates = torch.cat((mal_updates, all_updates), 0)

        agg_grads = tr_mean(mal_updates, n_attackers)

        loss = torch.norm(agg_grads - model_re)

        if prev_loss < loss:
            # print('successful lamda is ', lamda)
            lamda_succ = lamda
            lamda = lamda + lamda_fail / 2
        else:
            lamda = lamda - lamda_fail / 2

        lamda_fail = lamda_fail / 2
        prev_loss = loss

    mal_update = (model_re - lamda_succ * deviation)
    
    return mal_update

In [15]:
def init_weights(m):
    if isinstance(m, nn.Linear):
        torch.nn.init.xavier_uniform_(m.weight)
        m.bias.data.fill_(0.01)

torch.cuda.empty_cache()
use_cuda = torch.cuda.is_available()
criterion = nn.CrossEntropyLoss()
local_epochs = 2
batch_size = 16
num_workers = 100
local_lr = 0.01
global_lr = 1
nepochs = 50
nbyz = 20

all_data = torch.utils.data.ConcatDataset((trainset, testset))
fast_ndss = True
force = True
params = [.1, .2, .3, .4, .5, 10]

results = {}
for dir_p in np.array(params)[::-1]:
    results[dir_p] = []
    for run in range(3):
        print('===> Processing NDSS on Dir %.1f run %d' % (dir_p, run))
        each_worker_data, each_worker_label, each_worker_te_data, each_worker_te_label, global_test_data, global_test_label = get_client_data_dirichlet(all_data, num_workers, alpha=dir_p, force=force)
        best_global_acc=0
        epoch_num = 0

        fed_model = cnn().to(device)
        fed_model.apply(init_weights)
        model_received = []
        for i, (name, param) in enumerate(fed_model.state_dict().items()):
            model_received = param.view(-1).data.type(torch.cuda.FloatTensor) if len(model_received) == 0 else torch.cat((model_received, param.view(-1).data.type(torch.cuda.FloatTensor)))

        while epoch_num <= nepochs:
            torch.cuda.empty_cache()
            round_clients = np.arange(num_workers)
            round_benign = round_clients
            user_updates=[]
            benign_norm = 0

            for i in round_benign:
                model = copy.deepcopy(fed_model)
                optimizer = optim.SGD(model.parameters(), lr = local_lr, momentum=0.9, weight_decay=1e-4)
                for epoch in range(local_epochs):
                    train_loss, train_acc = train(
                        each_worker_data[i], torch.Tensor(each_worker_label[i]).long(), model, optimizer, batch_size)
                params = []
                for i, (name, param) in enumerate(model.state_dict().items()):
                    params = param.view(-1).data.type(torch.cuda.FloatTensor) if len(params) == 0 else torch.cat(
                        (params, param.view(-1).data.type(torch.cuda.FloatTensor)))
                update =  (params - model_received)
                benign_norm += torch.norm(update)/len(round_benign)
                user_updates = update[None,:] if len(user_updates) == 0 else torch.cat((user_updates, update[None,:]), 0)

            mal_update = ndss21_attack_trmean(user_updates[:nbyz], 5, dev_type='sign', threshold=20.0, threshold_diff=1e-5, fast_ndss=fast_ndss)
            user_updates[:nbyz] = torch.stack([mal_update]*nbyz)
            agg_update = tr_mean(user_updates, nbyz)

            del user_updates
            model_received = model_received + global_lr * agg_update
            fed_model = cnn().to(device)
            fed_model.apply(init_weights)
            start_idx=0
            state_dict = {}
            previous_name = 'none'
            for i, (name, param) in enumerate(fed_model.state_dict().items()):
                start_idx = 0 if i == 0 else start_idx + len(fed_model.state_dict()[previous_name].data.view(-1))
                start_end = start_idx + len(fed_model.state_dict()[name].data.view(-1))
                params = model_received[start_idx:start_end].reshape(fed_model.state_dict()[name].data.shape)
                state_dict[name] = params
                previous_name = name
            fed_model.load_state_dict(state_dict)
            val_loss, val_acc = test(global_test_data, global_test_label.long(), fed_model, criterion, use_cuda)
            is_best = best_global_acc < val_acc
            best_global_acc = max(best_global_acc, val_acc)

            if epoch_num%10==0 or epoch_num==nepochs-1:
                print('NDSS on Dir %.1f + TrMean | r %d e %d val loss %.3f val acc %.3f best val_acc %.3f'% (dir_p, run, epoch_num, val_loss, val_acc, best_global_acc))
            epoch_num+=1
        results[dir_p].append(best_global_acc)

for dir_p in params:
    print('======= Results for NDSS attack on Dir %.1f  =====' % dir_p)
    print(results[fcj_p])

===> Processing NDSS on Dir 10.0 run 0
generating participant indices for alpha 10.0
NDSS on Dir 10.0 + TrMean | r 0 e 0 val loss 1.043 val acc 69.790 best val_acc 69.790
NDSS on Dir 10.0 + TrMean | r 0 e 10 val loss 0.537 val acc 80.016 best val_acc 80.016
NDSS on Dir 10.0 + TrMean | r 0 e 20 val loss 0.489 val acc 82.346 best val_acc 82.346
NDSS on Dir 10.0 + TrMean | r 0 e 30 val loss 0.457 val acc 83.919 best val_acc 83.999
NDSS on Dir 10.0 + TrMean | r 0 e 40 val loss 0.463 val acc 84.357 best val_acc 84.815
NDSS on Dir 10.0 + TrMean | r 0 e 49 val loss 0.449 val acc 85.124 best val_acc 85.124
NDSS on Dir 10.0 + TrMean | r 0 e 50 val loss 0.446 val acc 85.253 best val_acc 85.253
===> Processing NDSS on Dir 10.0 run 1
generating participant indices for alpha 10.0
NDSS on Dir 10.0 + TrMean | r 1 e 0 val loss 1.138 val acc 67.629 best val_acc 67.629
NDSS on Dir 10.0 + TrMean | r 1 e 10 val loss 0.535 val acc 80.116 best val_acc 80.116
NDSS on Dir 10.0 + TrMean | r 1 e 20 val loss 0.4

NDSS on Dir 0.2 + TrMean | r 0 e 0 val loss 2.240 val acc 19.283 best val_acc 19.283
NDSS on Dir 0.2 + TrMean | r 0 e 10 val loss 0.974 val acc 62.769 best val_acc 62.769
NDSS on Dir 0.2 + TrMean | r 0 e 20 val loss 0.849 val acc 67.261 best val_acc 67.261
NDSS on Dir 0.2 + TrMean | r 0 e 30 val loss 0.833 val acc 68.516 best val_acc 68.516
NDSS on Dir 0.2 + TrMean | r 0 e 40 val loss 0.811 val acc 71.325 best val_acc 71.325
NDSS on Dir 0.2 + TrMean | r 0 e 49 val loss 0.796 val acc 72.490 best val_acc 72.490
NDSS on Dir 0.2 + TrMean | r 0 e 50 val loss 0.796 val acc 72.659 best val_acc 72.659
===> Processing NDSS on Dir 0.2 run 1
generating participant indices for alpha 0.2
NDSS on Dir 0.2 + TrMean | r 1 e 0 val loss 2.322 val acc 27.102 best val_acc 27.102
NDSS on Dir 0.2 + TrMean | r 1 e 10 val loss 1.015 val acc 62.888 best val_acc 62.888
NDSS on Dir 0.2 + TrMean | r 1 e 20 val loss 0.856 val acc 68.934 best val_acc 68.934
NDSS on Dir 0.2 + TrMean | r 1 e 30 val loss 0.820 val acc 

NameError: name 'fcj_p' is not defined

In [16]:
results

{10.0: [85.25341034087481, 85.31315346679565, 84.92482327745455],
 0.5: [80.94574411334298, 81.59283222902943, 81.05525133694032],
 0.4: [76.18763070875353, 76.69554823089105, 76.3370182260362],
 0.3: [74.48461307190932, 75.57016234126726, 74.77342892158508],
 0.2: [72.6593625619592, 71.48406374501992, 71.55378486055777],
 0.1: [60.4796974522293, 63.7937898089172, 63.853503184713375]}